# Analiza Wielowymiarowa - zajecia 8 -  Analiza dyskryminacji 

In [ ]:
from multidim.utils import resolve_stata, load_stata

STATA_PATH, STATA_TYPE = resolve_stata(version = 17, stype = "se")
# make sure they are proper ones
STATA_PATH, STATA_TYPE

In [ ]:
load_stata(STATA_PATH, STATA_TYPE)

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

Podczas dzisiejszych zajec wykorzystujemy dane udostepnione przez Uniwersytet Kalifornijski w Los Angles (UCLA)  
Dane i ich opis mozna znalezc na stronie https://stats.idre.ucla.edu/stata/dae/discriminant-function-analysis/


### Przykład 1.


Duży przewoźnik lotniczy zebrał informacje o osobach zatrudnionych na trzech stanowiskach
1. obsluga klienta (customer service personnel)
2. mechanik 
3. dyspozytorzy (dispatchers) 

Dyrektor Działu Personalnego chciałby wiedzieć, czy na każdym stanowisku pracują osoby o różnym typie osobowości.
Każdy pracownik został poddany serii testów psychologicznych obejmujących zainteresowanie aktywnoscią, 
mierzących towarzyskość pracownika oraz określających poziom jego konserwatyzmu


In [ ]:
%stata use https://stats.idre.ucla.edu/stat/stata/dae/discrim, clear

In [ ]:
discrim = pd.read_stata("https://stats.idre.ucla.edu/stat/stata/dae/discrim.dta")

In [ ]:
%stata des

In [ ]:
discrim.dtypes

 Rozpoczynamy analize od podstawowych statystyk zmiennych psychologicznych

In [ ]:
%stata summarize outdoor social conservative

In [ ]:
discrim.describe()

Podstawowe statystyki w podziale na typ wykonywanej pracy
- opcja stat definiuje statystki do wyswietlenia
- opcja col(stat) okresla ze maja one byc wyswietlone w kolumnach tabeli

In [ ]:
%stata tabstat outdoor social conservative, by(job) stat(n mean sd min max) col(stat)

In [ ]:
discrim.groupby(["job"])[["outdoor", "social", "conservative"]].describe().T

Tablica korelacji. Opcja sig wyswietla wartosc p dla hipotezy o braku korelacji

In [ ]:
%stata pwcorr outdoor social conservative, sig

In [ ]:
discrim[["outdoor", "social", "conservative"]].corr()

Analiza rozkladu zmiennej grupujacej


In [ ]:

%stata tabulate job

In [ ]:
tab = pd.DataFrame({"Freq": discrim["job"].value_counts()})
tab["Percent"] = tab / tab.sum() * 100
tab["Cum"] = tab["Percent"].cumsum()
tab

Kanoniczna analiza dyskryminacyjna  
Wykonanie polecenia powoduje wykonanie kanonicznej analizy dyskryminacyjnej  
Wyniki sa prezentowane w 6 tabelach  
Tabela 1 - wartosci wlasne i statystyczna istotnosc kierunkow dyskryminacji  
Tabela 2 - standaryzowane oszacowania kanonicznych wspolczynnikow dyskryminacji  
Tabela 3 - tabela ladunkow kanonicznych  
Tabela 4 - etykiety grup  
Tabela 5 - srednie wartosci zmiennych kanonicznych dla grup  
Tabela 6 - tabela klasyfikacji przy zalozonym jednostajnym rozkladzie a-priori.  
Przyjmowany rozklad a-priori mozna zmienic wykorzystujac opcje prior  

### LDA

In [ ]:
# Polecenie candisc" jest odpowiednikiem "discrim lda"
# Polecenie "candisc" wykonuje kanomiczną wersję liniowej analizy dykryminacyjnej (LDA).  
# Oba polecenia w Stata obliczają to samo, różnią się pod względem wyboru wyników w prezentacji. 

In [ ]:
%stata candisc outdoor social conservative, group(job)

In [ ]:
y = discrim.job
X = discrim.loc[:, ["outdoor", "social", "conservative"]]
lda = LinearDiscriminantAnalysis(solver = 'eigen')
lda.fit(X, y)
print(lda.explained_variance_ratio_)
print(confusion_matrix(y, lda.predict(X)))
print(classification_report(y, lda.predict(X)))

In [ ]:
%stata estat classfunctions

In [ ]:

print(lda.classes_)
print(lda.coef_.T, lda.intercept_)

In [ ]:
%%stata
/// Wyniki analizy mozna zaprezentowac na wykresie obrazujacym wartosci zmiennych kanonicznych
/// Dla przejrzystosci wykresu przed jego wykonaniem skracamy etykiety kategorii
label define job 1 "c" 2 "m" 3 "d", modify
scoreplot, msymbol(i)

/// Wykres rozrzutu standaryzowanych ladunkow czynnikowych funkcji dyskryminacyjnej
/// Pokazuje zaleznosc miedzy srednimi wartosciami oryginalnych zmiennych a wspolczynnikami korelacji kanonicznej
loadingplot

Te same wyniki mozna uzyskac poleceniem **discrim lda**
W przypadku tego polecenia Stata wyswietla wylacznie tablice klasyfikacji

In [ ]:
%stata discrim lda outdoor social conservative, group(job)

In [ ]:
# %stata help FUN postestimation
# %stata help discrim lda postestimation

In [ ]:
%%stata
/// Wyswietlenie tablicy analizy wariancji
estat anova
/// Tabela wartosci wlasnych i statystyczna istotnosc kierunkow dyskryminacji
estat canontest
/// Tabela oszacowan parametrow funkcji klasyfikacyjnych
estat classfunctions 
/// Tabela klasyfikacji
estat classtable
/// Tabela korelacji miedzy zmiennymi
estat correlations
/// Tabela kowariancji miedzy zmiennymi
estat covariance
/// Tabela standaryzowanych oszacowan kanonicznych wspolczynnikow dyskryminacji
estat loadings
/// Macierz struktury kanonicznej
estat structure

### Przykład 2.


Jest to przyklad analizy dyskryminacyjnej, ktory mozna znalezc w niemal kazdym 
podreczniku analizy wielowymiarowej. Zostal opracowany przez Fishera w 1936 roku
Analiza obejmuje trzy odmiany irysow i cztery zmienne opisujace ich cechy:
- Dlugosc platka [cm] (petal lenght)
- Szerokosc platka [cm] (petal width)
- Dlugosc listka kielicha [cm]
- Szerokosc listka kielicha [cm]

In [ ]:
from multidim.datasets import load_iris
iris = load_iris()
iris.columns = ["seplen", "sepwid", "petlen", "petwid", "iris"]
iris["iris"] = iris["iris"].astype("category")

Wykorzystujemy dane z pythona w STATA poprzez argument -d

In [ ]:
%%stata -d iris -force
di "Fisher example iris data"
egen giris = group(iris)

In [ ]:
from multidim.datasets import load_iris
iris = load_iris()
iris.columns = ["seplen", "sepwid", "petlen", "petwid", "iris"]
iris["iris"] = iris["iris"].astype("category")

In [ ]:

iris_x = iris.iloc[:,:-1].values
iris_y = iris.iloc[:,-1].values
# SCALING
# W pewnych przypadkach istotna bedzie standaryzacja
iris_x_scaled = (iris_x - np.mean(iris_x, axis = 0)) / np.std(iris_x, axis = 0)

In [ ]:
%stata des

In [ ]:
iris.dtypes

In [ ]:
%stata bysort giris: su seplen sepwid petlen petwid 

In [ ]:
iris.\
groupby("iris").\
describe().T

Analiza dyskryminacji

In [ ]:
%stata discrim lda seplen sepwid petlen petwid, group(giris)

In [ ]:
lda = LinearDiscriminantAnalysis(solver = 'eigen')
lda.fit(iris_x, iris_y)

Tabela nieprawidlowo klasyfikowanych obserwacji


In [ ]:

%stata estat list, misclassified

In [ ]:
# misclassified
np.where(iris_y != lda.predict(iris_x))

Wagi

In [ ]:
%stata estat classfunctions


In [ ]:

print(lda.classes_)
lda.coef_.T, lda.intercept_

/// Tabela niestandaryzowanych i standaryzowanych oszacowan kanonicznych wspolczynnikow dyskrminacji

In [ ]:
%stata estat loadings, unstandardized standardized


In [ ]:
lda.scalings_[:, 0:2]

In [ ]:
%stata loadingplot

In [ ]:
%stata scoreplot, msymbol(i)

In [ ]:
%%stata 
predict klasyfikacja
tab iris klasyfikacja

In [ ]:
confusion_matrix(iris_y, lda.predict(iris_x))

In [ ]:
# Tylko 2 pierwsze poziomy (odmiany skorczyka)
# 100 pierwszych obserwacji gdyz dane sa posortowane
# idx = list(range(100))
# lda = LinearDiscriminantAnalysis(n_components = 1, solver = 'eigen')
# lda.fit(iris_x_scaled[idx,:], iris_y[idx])
# Tutaj tylko jeden wektor wag
# print(lda.coef_.T, lda.intercept_)
# values = iris_x_scaled[idx,:] @ lda.coef_.T + lda.intercept_
# print(1/(1+np.exp(-values)))

### Dodatek Train/Test Prediciton

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris_x, iris_y, random_state=1234)

In [ ]:
lda = LinearDiscriminantAnalysis(n_components = 2)
lda.fit(X_train, y_train)
print(lda.score(X_test, y_test))

In [ ]:
# Predykcje
# lda.predict_proba(X_test)
# OR
# values = X_test @ lda.coef_.T + lda.intercept_
# values - values.max(axis=1)[:, np.newaxis]
# likelihood = np.exp(values - values.max(axis=1)[:, np.newaxis])
# predictions = np.round(likelihood / likelihood.sum(axis=1)[:, np.newaxis], 2)

In [ ]:
# Tablica klasyfikacji
confusion_matrix(y_test, lda.predict(X_test))

In [ ]:
print(classification_report(y_test, lda.predict(X_test)))

### Przykład 3.


Na podstawie danych z badania Diagnoza Spoleczna 2011 bedziemy analizowac stosunek obywateli do euro

In [ ]:
from multidim.datasets import load_euro
euro = load_euro()
euro["wiek"] = euro["wiek"].astype("int")

In [ ]:
%%stata -d euro -force
des

In [ ]:
%%stata
/// Zbadajmy rozklad zmiennej zaleznej
tab euro

In [ ]:
euro.dtypes

In [ ]:
euro.euro.value_counts()

In [ ]:
#%stata help discrim knn

In [ ]:
%%stata
/// Jaki model nalezy wybrac?
bysort euro: su wiek klm kobieta wyksz zna_angielski partia_PO partia_PiS
corr wiek klm kobieta wyksz zna_angielski partia_PO partia_PiS
/// Przeprowadz odpowiednia analize dyskryminacyjna
/// Zastanow sie jaki model zastosowac oraz jaki dorbac rozklad a-priori
/// Zastanow sie czy wszystkie zmienne dobrze rozdzielaja zbior

In [ ]:
euro[["wiek", "klm", "kobieta", "wyksz", "zna_angielski", "partia_PO", "partia_PiS", "euro"]].\
groupby("euro").\
describe().T

In [ ]:
euro.apply(lambda x: x if not hasattr(x, 'cat') else x.cat.codes).corr()

In [ ]:
%%stata
/// Zbadaj stosunek do euro metoda najblizszych sasiadow
// losujemy 10% probe prosta by przyspieszyc obliczenia
sample 10

In [ ]:
%%stata
egen geuro = group(euro)
egen gwyksz = group(wyksz)
egen gklm = group(klm)

Zastanow sie jaki rozklad a-priori wybrac aby zbadac poprawnosc klasyfikacji


In [ ]:
%%stata
discrim knn wiek gklm kobieta gwyksz zna_angielski partia_PO partia_PiS, group(geuro) k(5)

In [ ]:
# Bardzo szybki Algorytm - wiec wszytskie obserwacje
neigh = KNeighborsClassifier(n_neighbors=5)
X = pd.get_dummies(euro.loc[:, euro.columns != "euro"]).values
# X.columns = X.columns.str.replace("[^a-zA-Z\_0-9]", "", regex = True)
y = euro["euro"]
neigh.fit(X, y)

In [ ]:
preds = neigh.predict(X)

In [ ]:
#Tablica klasyfikacji 
confusion_matrix(y, preds)

In [ ]:
print(classification_report(y, preds))